성별, 나이, 키 3가지 변수만으로 머리너비와 얼굴수직길이를 예측한다. 

In [26]:
import numpy as np 
import pandas as pd 
from pycaret.regression import *    

## Create DataFrame

In [27]:
data_2015 = pd.read_excel('2015_7차_직접측정 데이터.xlsx')

In [28]:
data_2010 = pd.read_excel('./6th/2010_6차_직접측정 데이터.xlsx')

In [31]:
df1 = {'성별':[], '나이':[], '키':[], '머리너비':[], '얼굴수직길이':[]} 
df1['성별'] = data_2015['ⓞ_02_성별'] 
df1['나이'] = data_2015['ⓞ_06_나이_반올림']
df1['키'] = data_2015['①_003_키'] 
df1['머리너비'] = data_2015['①_115_머리너비'] 
df1['얼굴수직길이'] = data_2015['①_094_얼굴수직길이']   

In [32]:
df_2015 = pd.DataFrame(df1) 
df_2015.head()

,성별,나이,키,머리너비,얼굴수직길이
0,남,25,1736.0,159.0,118.0
1,남,28,1833.0,161.0,128.0
2,남,19,1744.0,162.0,127.0
3,남,20,1751.0,154.0,124.0
4,남,22,1851.0,158.0,117.0


In [33]:
df_2015 = df_2015.dropna()

In [36]:
df2 = {'성별':[], '나이':[], '키':[], '머리너비':[], '얼굴수직길이':[]} 
df2['성별'] = data_2010['성별'] 
df2['나이'] = data_2010['나이'] 
df2['키'] = data_2010['104:키'] 
df2['머리너비'] = data_2010['402:머리너비']  
df2['얼굴수직길이'] = data_2010['322:얼굴수직길이'] 

In [37]:
df_2010 = pd.DataFrame(df2) 
df_2010.head()

,성별,나이,키,머리너비,얼굴수직길이
0,남,23,1740,151,107
1,남,22,1722,160,109
2,남,24,1788,154,126
3,남,23,1770,159,115
4,남,23,1697,162,116


In [38]:
df_2010 = df_2010.dropna()

In [39]:
df_full = pd.concat([df_2015, df_2010], axis = 0)

In [41]:
df_full = df_full.dropna() 
df_full

,성별,나이,키,머리너비,얼굴수직길이
0,남,25,1736.0,159.0,118.0
1,남,28,1833.0,161.0,128.0
2,남,19,1744.0,162.0,127.0
3,남,20,1751.0,154.0,124.0
4,남,22,1851.0,158.0,117.0
...,...,...,...,...,...
7527,남,32,1796.0,154.0,129.0
7528,남,30,1700.0,169.0,121.0
7529,남,35,1826.0,156.0,118.0
7530,남,35,1662.0,163.0,121.0


In [42]:
df_full.to_csv("3_featues_predict.csv",index=False)

## Fit pycaret models for 머리너비

In [44]:
data = pd.read_csv("3_features_predict.csv")

In [45]:
reg = setup(data = data, target = '머리너비', session_id=123)

,Description,Value
0,session_id,123
1,Target,머리너비
2,Original Data,"(13949, 5)"
3,Missing Values,False
4,Numeric Features,3
5,Categorical Features,1
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(9764, 4)"


In [46]:
best = compare_models(sort = 'RMSE', n_select = 5) 

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,4.6880,35.9782,5.9972,0.3140,0.0380,0.0300,0.0930
lr,Linear Regression,4.7014,36.1481,6.0113,0.3107,0.0381,0.0301,0.5970
ridge,Ridge Regression,4.7014,36.1481,6.0113,0.3107,0.0381,0.0301,0.0050
lar,Least Angle Regression,4.7014,36.1481,6.0113,0.3107,0.0381,0.0301,0.0060
br,Bayesian Ridge,4.7013,36.1482,6.0113,0.3107,0.0381,0.0301,0.0060
lightgbm,Light Gradient Boosting Machine,4.7428,36.7514,6.0616,0.2990,0.0384,0.0303,0.5000
catboost,CatBoost Regressor,4.7467,36.7819,6.0639,0.2985,0.0384,0.0304,2.9240
xgboost,Extreme Gradient Boosting,4.8947,39.1106,6.2525,0.2543,0.0396,0.0313,0.3980
en,Elastic Net,4.9418,39.9247,6.3175,0.2390,0.0401,0.0316,0.0060
ada,AdaBoost Regressor,4.9853,40.3231,6.3487,0.2303,0.0403,0.0321,0.0370


## Blending Method

In [55]:
blended = blend_models(estimator_list = best, fold = 5)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,4.6189,34.2412,5.8516,0.3230,0.0373,0.0296
1,4.7623,37.4206,6.1172,0.3035,0.0386,0.0304
2,4.7027,36.0958,6.0080,0.3087,0.0381,0.0301
3,4.6724,36.3404,6.0283,0.2904,0.0380,0.0297
4,4.6843,35.7877,5.9823,0.3482,0.0380,0.0301
Mean,4.6881,35.9771,5.9975,0.3148,0.0380,0.0300
SD,0.0464,1.0275,0.0859,0.0197,0.0004,0.0003


In [56]:
pred_holdout = predict_model(blended)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,4.8170,37.9397,6.1595,0.3289,0.0391,0.0309


In [49]:
final_model = finalize_model(blended) 

In [50]:
final_model.estimators_

[GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                           init=None, learning_rate=0.1, loss='ls', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=123, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0, warm_start=False),
 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False),
 Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
       normalize=False, random_state=123, solver='auto', tol=0.001),
 Lars(copy_X=True, eps=2.220446049250313e-16, fit_intercept=True, fit_path=True,
      jitter=None, n_nonzero_

## Stacking Method

In [52]:
catboost = create_model('catboost')
stacker = stack_models(best, meta_model = catboost) 

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,4.6445,34.3820,5.8636,0.2868,0.0373,0.0298
1,4.6191,35.0173,5.9175,0.3389,0.0377,0.0296
2,4.8493,39.5525,6.2891,0.2780,0.0394,0.0309
3,4.8684,37.6819,6.1386,0.2838,0.0389,0.0312
4,4.7948,37.6518,6.1361,0.2827,0.0389,0.0308
5,4.7879,36.6007,6.0498,0.2933,0.0383,0.0305
6,4.8899,38.5527,6.2091,0.2675,0.0392,0.0311
7,4.6595,36.5968,6.0495,0.2648,0.0381,0.0297
8,4.7059,36.2981,6.0248,0.3208,0.0383,0.0301
9,4.6926,36.6977,6.0579,0.3462,0.0385,0.0302


In [57]:
## stacking with linear regression 
stacker = stack_models(best) 

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,4.6614,34.1148,5.8408,0.2923,0.0372,0.0299
1,4.6210,34.8675,5.9049,0.3417,0.0376,0.0296
2,4.8083,38.9083,6.2377,0.2898,0.0391,0.0306
3,4.7191,35.6493,5.9707,0.3224,0.0379,0.0302
4,4.7057,36.2159,6.0180,0.3100,0.0382,0.0302
5,4.7299,36.0830,6.0069,0.3033,0.0380,0.0302
6,4.7796,37.1671,6.0965,0.2938,0.0385,0.0304
7,4.5463,35.3049,5.9418,0.2908,0.0374,0.0289
8,4.7110,35.5665,5.9638,0.3345,0.0379,0.0302
9,4.6524,35.8897,5.9908,0.3606,0.0381,0.0299


It appears that the blending ensemble has a slightly better RMSE of 5.99mm. 
However it is higher than our previously recorded RMSE of 4.60 mm, when we used a simple catboost model with 5 features instead of 3. 